# Convolution

### Apply convolutions to arrays, functions and images

In [ ]:
import numpy as np
from scipy import ndimage, signal as sg
from skimage import data, filters, io, color
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
#Default value for color map is "jet", change to gray 
plt.rcParams["image.cmap"] = "gray"
#change default nterpolation tp nearest
plt.rcParams["image.interpolation"] = "nearest"

### 1d convolution - NumPy

In [ ]:
f =[1,5,2,4]
g=[2,3,1]

In [ ]:
#Convolution operator flips the second array before “sliding” the two across one another.
np.convolve(f, g, "full")

In [ ]:
#Only two position where they completely overlap
np.convolve(f, g, "valid")

In [ ]:
#Only return the middle values of the convolution
np.convolve(f, g, "same")

### 2d convolution - Scipy

In [ ]:
sg.convolve([[255, 7, 3],[212, 240, 4],[218, 216, 230]],[[2, -1]], "valid")

### Signal convolution

Convolve a signal with a square pulse

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(10,9))

t = np.linspace(0, 1, 1000, endpoint=False)
#sine wave
y = np.sin(2*2*np.pi*t)
ax1.plot(t, y, 'r')
ax1.set_title('Sine Signal')

#square pulse
ts = sg.square(2 * np.pi * 5 * t)
ax2.plot(t, ts, 'b')
ax2.set_title('Square Pulse')

#convolved output
ax3.plot(sg.convolve(y ,ts, "same"),'g')
ax3.set_title('Convolved Signal')
plt.show()

### Gaussian Convolution

The Gaussian smoothing operator is a 2-D convolution operator as a point-spread function. It is used to "blur" images and remove details and noise. It is similar to the mean filter, but it uses a different kernel that represents the shape of a Gaussian hump. 

The Gaussian outputs a weighted average of each pixel's neighborhood, with the average weighted more towards the value of the central pixels. This is in contrast to the mean filter's uniformly weighted average. Because of this, a Gaussian provides gentler smoothing and preserves edges better than a similarly sized mean filter.  

In [ ]:
def gauss_kern(size, sizey=None):
    """ Returns a normalized 2D gauss kernel array for convolutions """
    size = int(size)    
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)               
    #print size, sizey    
    x, y = np.mgrid[-size:size+1, -sizey:sizey+1]
    g = np.exp(-(x**2/float(size)+y**2/float(sizey)))
    return g / g.sum()

def blur_image(im, n, ny=None) :
    """ blurs the image by convolving with a gaussian kernel of typical
        size n. The optional keyword argument ny allows for a different
        size in the y direction.
    """
    g = gauss_kern(n, sizey=ny)
    improc = sg.convolve(im,g, mode='valid')
    return(improc)    


xmin, xmax, ymin, ymax = -70, 70, -70, 70
extent = xmin, xmax, ymin, ymax

X, Y = np.mgrid[-70:70, -70:70]
print (X.shape)

#circlular rings
Z1 = np.cos((X**2+Y**2)/200.)

#circlular rings + gausian noise
Z2 = np.cos((X**2+Y**2)/200.)+ np.random.normal(size=X.shape)    

#Plot the images
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(20, 8))

#Original Signal
ax1.imshow(abs(Z1), cmap=cm.jet, alpha=0.9, interpolation='bilinear', extent=extent)
ax1.set_title(r'Original Signal')

#Signal with Noise
ax2.imshow(abs(Z2), cmap=cm.jet, alpha=.9, interpolation='bilinear', extent=extent)
ax2.set_title(r'Signal + Noise')

#Gausian Kernel
P = gauss_kern(3)
ax3.imshow(abs(P), cmap=cm.jet, alpha=.9, interpolation='bilinear', extent=extent)
ax3.set_title(r'Gaussian Filter')

#Blurred Image recovery after applying Gausian Kernel
U = blur_image(Z2, 3)
ax4.imshow(abs(U), cmap=cm.jet, alpha=.9, interpolation='bilinear', extent=extent)
ax4.set_title(r'Cleaned')

plt.show()

### Convolution applied to images

In [ ]:
image = data.camera()
io.imshow(image)
io.show()

In [ ]:
pixelated = image[::3, ::3]
io.imshow(pixelated)
io.show()

### Mean Kernel

Mean filter (also known as an averaging filter) is a type of image filtering that replaces each pixel's value with the average value of its neighboring pixels

In [ ]:
mean_kernel = 1.0/9.0 * np.ones((3, 3))
print (mean_kernel)
image1 = ndimage.convolve(pixelated, mean_kernel, cval=1.0)
io.imshow(image1)
io.show()

### Smooth Kernel

Designed for a smoothing effect, this kernel emphasizes the contribution of the center pixel while still considering the contributions of its surrounding pixels

In [ ]:
#What do the numbers in the following convolution kernel tell about it ?
smooth_kernel = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])
print(smooth_kernel)
image2 = ndimage.convolve(pixelated, smooth_kernel, cval=1.0)
io.imshow(image2)

### Gaussian Kernel

In [ ]:
kernel_size = 5
sigma = 1.0

# Apply Gaussian blur
image3 = ndimage.convolve(pixelated, gauss_kern(kernel_size), cval=1.0)
io.imshow(image3)

### Edge Detection Kernel

In [ ]:
edge_kernel = np.array([[-1,-1,-1],[-1,7,-1],[-1,-1,-1]])
#edge_kernel = np.array([[-1/8,-1/8,-1/8],[-1/8,1,-1/8],[-1/8,-1/8,-1/8]])
print(edge_kernel)

image4 = ndimage.convolve(image, edge_kernel, cval=1.0)
io.imshow(image4)
io.show()

### Emboss Kernel

The emboss kernel (similar to the sobel kernel) givens the illusion of depth by emphasizing the differences of pixels in a given direction

In [ ]:
emboss_kernel = np.array([[2,0,0],[0,-1,0],[0,0,-1]])
print(emboss_kernel)
image6 = ndimage.convolve(image, emboss_kernel, cval=1.0)
io.imshow(image6)
io.show()

### Sharpen kernel

The sharpen kernel emphasizes differences in adjacent pixel values.

In [ ]:
sharp_kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]]) #High-Pass Filter
#sharp_kernel = np.array([[0, -1, 0],[-1, 4, -1],[0, -1, 0]]) #laplacian

print(sharp_kernel)
image6 = ndimage.convolve(image, sharp_kernel)
io.imshow(image6)
io.show()